In [ ]:
import pandas as pd
import numpy as np
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os 
os.chdir('/content/drive/My Drive/Assignment')
!ls

processed_train.csv  RFC_Model.pckl    Untitled.ipynb
RFC_Model_all.pckl   test_indessa.csv  XGB.pckl


In [ ]:
#import columns to be dropped from data discovery 

columns_to_drop =['member_id', 'desc', 'mths_since_last_record', 'mths_since_last_major_derog', 'verification_status_joint', 'mths_since_last_delinq','batch_enrolled' ]

In [ ]:
test_data= pd.read_csv('test_indessa.csv')

In [ ]:
test_data.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,application_type,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,term_int,sub_grade_int,last_week_pay_int
0,14000,14000,14000.0,60 months,16.24,C,C5,Data Analyst,4,RENT,73500.0,Source Verified,n,credit_card,450xx,OH,28.65,1.0,1.0,16.0,0.0,29828,65.3,53.0,f,3915.61,0.0,0.0,0.0,0.0,INDIVIDUAL,104th week,0.0,0.0,85230.0,45700.0,60,5,104
1,16000,16000,16000.0,60 months,9.49,B,B2,Senior Database Administrator,5,MORTGAGE,105000.0,Source Verified,n,major_purchase,275xx,NC,31.19,0.0,0.0,8.0,0.0,4161,19.4,63.0,w,1495.06,0.0,0.0,0.0,0.0,INDIVIDUAL,57th week,0.0,0.0,444991.0,21400.0,60,2,57
2,11050,11050,11050.0,60 months,15.61,D,D1,Customer service representative,10,RENT,62300.0,Not Verified,n,debt_consolidation,672xx,KS,21.73,1.0,0.0,11.0,1.0,12064,74.0,20.0,w,2096.21,0.0,0.0,0.0,0.0,INDIVIDUAL,70th week,0.0,0.0,105737.0,16300.0,60,1,70
3,35000,35000,34700.0,60 months,12.69,C,C2,ACCT OFFICER,10,OWN,150000.0,Source Verified,n,debt_consolidation,961xx,CA,17.27,0.0,0.0,21.0,0.0,32549,45.0,26.0,f,1756.31,0.0,0.0,0.0,0.0,INDIVIDUAL,22th week,0.0,0.0,287022.0,72400.0,60,2,22
4,6500,6500,6500.0,36 months,6.89,A,A3,Paralegal,5,MORTGAGE,52000.0,Not Verified,n,major_purchase,707xx,LA,20.19,0.0,0.0,16.0,0.0,5133,19.2,36.0,w,172.21,0.0,0.0,0.0,0.0,INDIVIDUAL,22th week,0.0,0.0,234278.0,26700.0,36,3,22


In [ ]:
#drop the columns
#dropping title as well, because of less time , not running sentiment analysis
test_data.drop(['member_id','desc', 'mths_since_last_record', 'mths_since_last_major_derog', 'verification_status_joint', 
           'mths_since_last_delinq','batch_enrolled','title' ],inplace=True,axis=1 )

In [ ]:
#define features 
features = test_data.columns
print(features, features.shape)

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'pymnt_plan', 'purpose',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'collection_recovery_fee', 'collections_12_mths_ex_med',
       'application_type', 'last_week_pay', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'total_rev_hi_lim'],
      dtype='object') (36,)


In [ ]:
#lets preprocess categorical columns first 
var_cat = []
for var in features:
    if test_data[var].dtypes == 'O':
        var_cat.append(var)
print(var_cat)

['term', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'pymnt_plan', 'purpose', 'zip_code', 'addr_state', 'initial_list_status', 'application_type', 'last_week_pay']


In [ ]:
(test_data[var_cat].isnull().mean()*100)

term                   0.000000
grade                  0.000000
sub_grade              0.000000
emp_title              5.811788
emp_length             5.052528
home_ownership         0.000000
verification_status    0.000000
pymnt_plan             0.000000
purpose                0.000000
zip_code               0.000000
addr_state             0.000000
initial_list_status    0.000000
application_type       0.000000
last_week_pay          0.000000
dtype: float64

In [ ]:
random_impute= ['emp_length','emp_title']

In [ ]:
#single go pandas is going heavy on data , so running it in two chunks 
def random_imputatiochunk1(data, column):
    
    n=len(data)//2
    for x in range(0,n):
        if pd.isna(data[column][x]):
            #print(data['emp_length'][x])
            data[column][x]=np.random.choice(data[column].dropna().values)
            
def random_imputatiochunk2(data, column):
    
    n=len(data)//2
    for x in range(n,354951):
        if pd.isna(data[column][x]):
            #print(data['emp_length'][x])
            data[column][x]=np.random.choice(data[column].dropna().values)

In [ ]:
for i in random_impute:
    random_imputatiochunk1(test_data,i)
    random_imputatiochunk2(test_data,i)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [ ]:
test_data['emp_title'].isna().sum()

0

In [ ]:
#process Column with str+int
def to_int(data,column):
    data[column+'_int'] = data[column].str.extract('(\d+)').astype(int) 

In [ ]:
to_int_columns= ['term','sub_grade','last_week_pay']

In [ ]:
# last_week_pay has value NAth week , which needs to be converted into int+str
#this values can be handled by random imputation , for now replaced by 0
test_data['last_week_pay'].replace({'NAth week':'0th week'}, inplace=True)

In [ ]:
for i in to_int_columns:
    print(i)
    to_int(test_data,i)

term
sub_grade
last_week_pay


In [ ]:
#lable encode emp length 
coding_dict= {'< 1 year':0, '1 year':1, '2 years':2, '9 years':9, '10+ years':10, '5 years':5,
       '8 years':8, '7 years':7, '4 years':4, '3 years':3, '6 years':6  }
test_data['emp_length']= test_data['emp_length'].map(coding_dict)

In [ ]:
#columns to be lable encoded 
tobe_lable_encode= ['grade ', 'home_ownership ', 'verification_status ', 'pymnt_plan ', 
                    'purpose ',  'addr_state ', 'initial_list_status',  'application_type ' ]

In [ ]:
data=pd.read_csv('processed_train.csv')

In [ ]:
for feat in ['grade', 'home_ownership']:

    lbe = LabelEncoder()
    lbe.fit(data[feat].values)
    diz_map_train = dict(zip(lbe.classes_, lbe.transform(lbe.classes_)+1))

    for i in set(test_data[feat]).difference(data[feat]):
        diz_map_train[i] = np.nan

    data[feat] = [diz_map_train[i] for i in data[feat].values]
    test_data[feat] = [diz_map_train[i] for i in test_data[feat].values]

In [ ]:
#use sklearn lable encoder 
#using fit for train data and transform for test data
grade= LabelEncoder()
data['grade']= grade.fit(data['grade'])
test_data['grade']= grade.transform(test_data['grade'])

home_ownership= LabelEncoder()
data['home_ownership']= home_ownership.fit(data['home_ownership'])
test_data['home_ownership']= home_ownership.transform(test_data['home_ownership'])
    
verification_status= LabelEncoder()
data['verification_status']= verification_status.fit(data['verification_status'])
test_data['verification_status']= verification_status.transform(test_data['verification_status'])

pymnt_plan= LabelEncoder()
data['pymnt_plan']= pymnt_plan.fit(data['pymnt_plan'])
test_data['pymnt_plan']= pymnt_plan.transform(test_data['pymnt_plan'])

purpose= LabelEncoder()
data['purpose']= purpose.fit(data['purpose'])
test_data['purpose']= purpose.transform(test_data['purpose'])

addr_state= LabelEncoder()
data['addr_state']= addr_state.fit(data['addr_state'])
test_data['addr_state']= addr_state.transform(test_data['addr_state'])

initial_list_status= LabelEncoder()
data['initial_list_status']= initial_list_status.fit(data['initial_list_status'])
test_data['initial_list_status']= initial_list_status.transform(test_data['initial_list_status'])

application_type= LabelEncoder()
data['application_type']= application_type.fit(data['application_type'])
test_data['application_type']= application_type.transform(test_data['application_type'])

ValueError: ignored

In [ ]:
#mean target encoding 

pur= data.groupby('purpose')['loan_status'].mean()
data['purpose']= data['purpose'].map(pur)

In [ ]:
data.drop(['sub_grade','term','last_week_pay'], axis=1, inplace=True)

In [ ]:
#coun frequency encoding for emp_title

emp_ttl= data.groupby('emp_title')['loan_status'].count()

In [ ]:
data['emp_title']= data['emp_title'].map(emp_ttl)

In [ ]:
##preprocessign Numerical data 


In [ ]:
## as per correlation matrix , we can delete some columns
columns_to_drop = ['loan_amnt', 'funded_amnt','total_rev_hi_lim','open_acc','collection_recovery_fee']

In [ ]:
data.drop(['loan_amnt', 'funded_amnt','total_rev_hi_lim','open_acc','collection_recovery_fee' ],inplace=True,axis=1 )

In [ ]:
features_n= data.columns
var_num = [] 
for var in features_n: 
    if data[var].dtypes != 'O': 
        var_num.append(var) 
print(var_num)

['funded_amnt_inv', 'int_rate', 'grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collections_12_mths_ex_med', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'loan_status', 'term_int', 'sub_grade_int', 'last_week_pay_int']


In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(data[var_num])
x=imp.transform(data[var_num])
data[var_num]= pd.DataFrame(x)

In [ ]:
data.isna().sum()

funded_amnt_inv               0
int_rate                      0
grade                         0
emp_title                     0
emp_length                    0
home_ownership                0
annual_inc                    0
verification_status           0
pymnt_plan                    0
purpose                       0
zip_code                      0
addr_state                    0
dti                           0
delinq_2yrs                   0
inq_last_6mths                0
pub_rec                       0
revol_bal                     0
revol_util                    0
total_acc                     0
initial_list_status           0
total_rec_int                 0
total_rec_late_fee            0
recoveries                    0
collections_12_mths_ex_med    0
application_type              0
acc_now_delinq                0
tot_coll_amt                  0
tot_cur_bal                   0
loan_status                   0
term_int                      0
sub_grade_int                 0
last_wee

In [ ]:
data.head()
#data['term_int'] = data['term'].str.extract('(\d+)').astype(int)

,funded_amnt_inv,int_rate,grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,...,recoveries,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,loan_status,term_int,sub_grade_int,last_week_pay_int
0,14350.0,19.19,4,319.0,9.0,4,28700.0,1,0,1,...,0.0,0.0,0,0.0,0.0,28699.0,0.0,36.0,3.0,26.0
1,4800.0,10.99,1,76.0,0.0,1,65000.0,1,0,2,...,0.0,0.0,0,0.0,0.0,9974.0,0.0,36.0,4.0,9.0
2,10000.0,7.26,0,2309.0,2.0,4,45000.0,0,0,1,...,0.0,0.0,0,0.0,65.0,38295.0,0.0,36.0,4.0,9.0
3,15000.0,19.72,3,1.0,10.0,5,105000.0,0,0,1,...,0.0,0.0,0,0.0,0.0,55564.0,0.0,36.0,5.0,135.0
4,16000.0,10.64,1,1.0,10.0,5,52000.0,2,0,0,...,0.0,0.0,0,0.0,0.0,47159.0,0.0,36.0,2.0,96.0


In [ ]:
data.to_csv('processed_train.csv', encoding='utf-8',index=False)